# Delta Hedging in the Binomial Model

## 1. Binomial Model

Fundamental Information:
- Stock price between timesteps can either move up or down 
- Up and Downmovements have a certain probability

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

S_0 = 100   # Initial stock price
u = 1.1     # Up factor
d = 0.9     # Down factor
p = 0.6     # Probability of moving
N = 10      # Number of time steps

prices = [S_0]

for _ in range(N):
    move = np.random.choice([u,d],p=[])
    prices.append(prices[-1] * move)

